In [28]:
import numpy as np
import pandas as pd

In [29]:
def get_params(df, class_column):
    """
    This functions returns a dictionary with counts for every class in a ground truth column.
    :param df: a pandas dataframe with a column of ground truth predictions.
    :param class_column: string with the name of the ground truth column.
    :return : a dictionary where keys are classes and values are counts.
    """
    N = dict(df[class_column].value_counts())
    return N


def get_random_weights(size, equal_weight=False):
    """
    This functions returns a numpy array with weights that sum to 1.
    :param size: how many weights to have.
    :param equal_weight: if True all weights are equal in value.
    :return : a numpy array with weights between 0 and 1 which sum to 1.
    """
    if equal_weight:
        w = np.ones(size)
        w_sum = sum(w)
        w = w/w_sum
    else:
        w = np.random.randint(low=0, high=10, size=size)
        w_sum = sum(w)
        w = w/w_sum
    return w
  

def scale_predicted_probs(y_pred_1, y_pred_2):
    """
    Given two numbers this function returns their weighted values.
    """
    sum_ = y_pred_1 + y_pred_2
    y_pred_1, y_pred_2 = y_pred_1/sum_, y_pred_2/sum_
    return y_pred_1, y_pred_2


def cap_prob(prob):
    """
    This function caps values. If it is 1 it returns a very close number to 1 (e.g: 0.999...) and if it is 0
    it returns a number very close to 0 (e.g: 0.000...1). This helps prevent extreme values when using a logarithm.
    """
    p = max(min(prob, 1-10**-15),10**-15)
    return p


def sep():
    print("-"*100)

In [30]:
def multi_class_logarithmic_loss(y_true, y_pred_1, y_pred_0, N, w):
    # Scale row predictions for each class
    y_pred_1, y_pred_2 = scale_predicted_probs(y_pred_1, y_pred_0)
    if y_true == 1:
        # Cap predicted probabilities
        y_pred_1 = cap_prob(y_pred_1)
        # Apply natural log to each probability   
        y_pred_1_log = np.log(y_pred_1)
        # Get y_N
        y_N = 1/N["A"]
        mcll = w * y_N * y_pred_1_log
    else:
        y_pred_0 = cap_prob(y_pred_0)
        y_pred_0_log = np.log(y_pred_0)
        y_N = 1/N["B"]
        mcll = (1-w) * y_N * y_pred_0_log
    return -mcll

In [31]:
y_true = ["A","A","B","B","B"]
y_pred_A = [0.05, 0.11, 0.21, 0.48, 0.18]
y_pred_B = [0.20, 0.80, 0.23, 0.42, 0.10]
df = pd.DataFrame([y_true, y_pred_A, y_pred_B]).T
df.columns = ["y_true", "y_pred_A", "y_pred_B"]
df.head()

,y_true,y_pred_A,y_pred_B
0,A,0.05,0.2
1,A,0.11,0.8
2,B,0.21,0.23
3,B,0.48,0.42
4,B,0.18,0.1


In [32]:
N = get_params(df, "y_true")
print(f"N is: {N}")
sep()
w = get_random_weights(2, equal_weight=True)
print(f"Weights are: {w}")
sep()
# Assume A=1 and B=0
mcll = multi_class_logarithmic_loss(1, 0.5, 0.7, N, 2.0)
print(f"Weighted multi-class logarithmic loss: {mcll}")

N is: {'B': 3, 'A': 2}
----------------------------------------------------------------------------------------------------
Weights are: [0.5 0.5]
----------------------------------------------------------------------------------------------------
Weighted multi-class logarithmic loss: 0.8754687373538999
